In [2]:
import pandas as pd
raw_df = pd.read_csv("../data/NYC.csv") #load data

In [3]:
df=raw_df.copy()

In [4]:
df.columns #check columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'],
      dtype='object')

In [5]:
df.info() #give column names, data types, null values and memory usage. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


In [6]:
df.head() #peek first 5 rows

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [8]:
df.columns = df.columns.str.lower() #convert column names to lower case for consistency

In [9]:
df.shape #to know dataset size in rows and columns

(1458644, 11)

In [11]:
columns_to_keep =[
    'id',
    'pickup_datetime',
    'passenger_count',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'trip_duration'
] #cleaning the raw data by eliminating the unncecessary columns

In [12]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime']) #converting the time from string to timeobject

In [13]:
df['pickup_hour'] = df['pickup_datetime'].dt.hour #converting the timestamp to hour

In [14]:
df['pickup_day'] = df['pickup_datetime'].dt.day_name() #storing the day 

In [15]:
df['trip_duration'] = df['trip_duration']/60 #converting the trip duration to minutes

In [16]:
df = df[df['trip_duration'] < 120] #remove outliers which has trip duration more than 2 hrs

In [17]:
trip_facts = df[['id','pickup_hour','pickup_day',
                 'passenger_count','trip_duration']] #creating table with columns id,pickup hour, pickupday, passenger count and tripduration

In [47]:
location_dim = df[['id','pickup_latitude','pickup_longitude']] #creating table with columns id, pickup lat, pickup long

In [48]:
trip_facts.to_csv("trip_facts.csv", index=False)
location_dim.to_csv("location_dim.csv", index=False) #exporting tables to csv

In [27]:
import sqlite3
conn = sqlite3.connect("taxi_analysis.db") #importing sqlite

In [28]:
trip_facts.to_sql("trip_facts", conn, if_exists='replace', index=False)
location_dim.to_sql("location_dim", conn, if_exists='replace', index=False) #pushing tables to database

In [29]:
pd.read_sql("SELECT * FROM trip_facts LIMIT 5;", conn) #checking tables exist

,id,pickup_hour,pickup_day,passenger_count,trip_duration
0,id2875421,17,Monday,1,7.583333
1,id2377394,0,Sunday,1,11.050000
2,id3858529,11,Tuesday,1,35.400000
3,id3504673,19,Wednesday,1,7.150000
4,id2181028,13,Saturday,1,7.250000


In [30]:
def demand_bucket(hour):
    if 7 <= hour <= 10 or 17 <= hour <= 19:
        return "Peak"
    elif 11 <= hour <= 16:
        return "Mid-Day"
    else:
        return "Off-Peak"

df['demand_period'] = df['pickup_hour'].apply(demand_bucket) #creating new column of demand period and categoring them as peak, miday, offpeak

In [32]:
df['demand_period'].value_counts() 

Off-Peak    522405
Peak        512483
Mid-Day     421503
Name: demand_period, dtype: int64

In [33]:
trip_facts.to_csv("trip_facts.csv", index=False) #re-export

In [34]:
trip_facts.to_sql("trip_facts", conn, if_exists='replace', index=False) #re-load to sqlite

In [35]:
query = """
SELECT pickup_hour, COUNT(*) as total_trips
FROM trip_facts
GROUP BY pickup_hour
ORDER BY total_trips DESC;
"""

peak_demand_hours = pd.read_sql(query, conn)

peak_demand_hours.head() #peak demand hours

,pickup_hour,total_trips
0,18,90456
1,19,90193
2,21,84060
3,20,83946
4,22,80343


In [36]:
query = """
SELECT 
    pickup_day,
    COUNT(*) AS total_trips
FROM trip_facts
GROUP BY pickup_day
ORDER BY total_trips DESC;

"""

trips_by_day = pd.read_sql(query, conn)

trips_by_day.head() #trips by day

,pickup_day,total_trips
0,Friday,223167
1,Saturday,220455
2,Thursday,218262
3,Wednesday,209860
4,Tuesday,202452


In [57]:
df[['pickup_hour','demand_period']].head()

,pickup_hour,demand_period
0,17,Peak
1,0,Off-Peak
2,11,Mid-Day
3,19,Peak
4,13,Mid-Day


In [38]:
trip_facts = df[['id','pickup_hour','pickup_day',
                 'passenger_count','trip_duration','demand_period']]

trip_facts.to_sql("trip_facts", conn, if_exists='replace', index=False)

In [39]:
pd.read_sql("PRAGMA table_info(trip_facts);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,TEXT,0,None,0
1,1,pickup_hour,INTEGER,0,None,0
2,2,pickup_day,TEXT,0,None,0
3,3,passenger_count,INTEGER,0,None,0
4,4,trip_duration,REAL,0,None,0
5,5,demand_period,TEXT,0,None,0


In [40]:
query = """
SELECT 
    demand_period,
    COUNT(*) as total_trips,
    ROUND(AVG(trip_duration),2) as avg_trip
FROM trip_facts
GROUP BY demand_period;
"""

avg_trip_duration = pd.read_sql(query, conn)

avg_trip_duration.head() #average trip duration

,demand_period,total_trips,avg_trip
0,Mid-Day,421503,15.33
1,Off-Peak,522405,12.75
2,Peak,512483,14.00


In [64]:
df.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'pickup_hour', 'pickup_day', 'demand_period'],
      dtype='object')

In [66]:
trip_facts = df[['id',
                 'pickup_hour',
                 'pickup_day',
                 'passenger_count',
                 'trip_duration',
                 'demand_period']]

In [67]:
trip_facts.head()

,id,pickup_hour,pickup_day,passenger_count,trip_duration,demand_period
0,id2875421,17,Monday,1,7.583333,Peak
1,id2377394,0,Sunday,1,11.050000,Off-Peak
2,id3858529,11,Tuesday,1,35.400000,Mid-Day
3,id3504673,19,Wednesday,1,7.150000,Peak
4,id2181028,13,Saturday,1,7.250000,Mid-Day


In [68]:
trip_facts.to_csv("trip_facts.csv", index=False)

In [42]:
query = """
SELECT 
    passenger_count,
    COUNT(*) AS total_trips
FROM trip_facts
GROUP BY passenger_count
ORDER BY total_trips DESC;

"""

passanger_distribution = pd.read_sql(query, conn)
passanger_distribution.head() #Passenger Distribution

,passenger_count,total_trips
0,1,1032177
1,2,209981
2,5,77858
3,3,59771
4,6,48197


In [44]:
query = """
SELECT 
    pickup_hour,
    COUNT(*) AS total_trips,
    ROUND(AVG(trip_duration),2) AS avg_duration
FROM trip_facts
GROUP BY pickup_hour
ORDER BY pickup_hour;
"""

demand_by_duration = pd.read_sql(query, conn)

demand_by_duration.head() #Demand + Duration


,pickup_hour,total_trips,avg_duration
0,0,53171,13.01
1,1,38493,12.32
2,2,27909,11.69
3,3,20846,11.70
4,4,15753,12.21


In [49]:
sample_df = df.sample(5000, random_state=42)
sample_df.to_csv("trip_sample.csv", index=False)